# Applying relational learning to time series: Occupancy detection

The purpose of this notebook is to demonstrate how relational learning can be applied to a simple time series data set.

Many data scientists are surprised to learn that **relational learning is great for time series**.

But once you think about it, it is not that surprising. When we handle time series data, we usually extract features like this:

1. **Aggregations over time**, such as the average value of some column for the last 3 days.

2. **Seasonal effects**, such as today is a Wednesday, so let's get the average value for the last four Wednesdays.

3. **Lag variables**, such as get the value of some column from two hours ago.

Using relational learning, we can **extract all of these features automatically**. We can then apply state-of-the-art machine learning algorithms, like xgboost. Usually, this performs **better than traditional time series analysis**.

To show how relational learning can be used for time series, we apply a relational learning algorithm to a very **simple public domain time series dataset**. You can download the dataset from [here](http://archive.ics.uci.edu/ml/datasets/Occupancy+Detection+) (but you don't have to - downloading the data is integrated in this notebook).

As a **reference and benchmark**, we use [this paper](http://www.worldresearchlibrary.org/up_proc/pdf/568-148612088816-20.pdf):

* Accurate occupancy detection of an office room from light, temperature, humidity and CO2 measurements using statistical learning models. Luis M. Candanedo, Veronique Feldheim. Energy and Buildings. Volume 112, 15 January 2016, Pages 28-39.

This notebook is **completely self-contained**. You can just run it to reproduce our results. You will need [getML](https://getml.com/product) in order to do so. You can [download it for free](https://getml.com/product).

If you want to learn more about getML, check out the [official documentation](https://docs.getml.com/latest/).

## The challenge

The challenge is simple: We want to **predict whether an office room is occupied** at a given moment in time using sensor data. The data is measured about **once a minute**.

Here are the available columns:

* Date, year-month-day hour:minute:second
* Temperature, in Celsius
* Relative Humidity, %
* Light, in Lux
* CO2, in ppm
* Humidity Ratio, Derived quantity from temperature and relative humidity, in kgwater-vapor/kg-air
* **Target**: Occupancy, 0 or 1, 0 for not occupied, 1 for occupied status

In [1]:
import os
from urllib import request

import getml
import numpy as np
import pandas as pd

## Getting the data

This notebook is completely self-contained. So we will help you get the data:

In [2]:
fnames = [
    'datatraining.txt',
    'datatest.txt',
    'datatest2.txt'
]

for fname in fnames:
    if not os.path.exists(fname):
        fname, res = request.urlretrieve(
            "https://raw.githubusercontent.com/LuisM78/Occupancy-detection-data/master/" + fname, 
            fname
        )


We now set the project.

In getML, every data frame and model is **tied to a project**. If you change the project, then the memory is flushed and all unsaved changes are lost (but don't worry, models are saved automatically).

In [3]:
getml.engine.set_project('occupancy_detection')

Creating new project 'occupancy_detection'


## Loading data

First, we **load the data into pandas**. There are other ways to do this, but this is such a small data set that pandas seems the easiest approach. 

The data set is conveniently separated into a training, a validation and a testing set. This allows us to **directly benchmark our results** against the results of the original paper.

This is what the data looks like:

In [4]:
datatraining_pandas = pd.read_csv('datatraining.txt')
datatest_pandas = pd.read_csv('datatest.txt')
datatest2_pandas = pd.read_csv('datatest2.txt')

datatraining_pandas

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-04 17:51:00,23.18,27.2720,426.0,721.250000,0.004793,1
2,2015-02-04 17:51:59,23.15,27.2675,429.5,714.000000,0.004783,1
3,2015-02-04 17:53:00,23.15,27.2450,426.0,713.500000,0.004779,1
4,2015-02-04 17:54:00,23.15,27.2000,426.0,708.250000,0.004772,1
5,2015-02-04 17:55:00,23.10,27.2000,426.0,704.500000,0.004757,1
...,...,...,...,...,...,...,...
8139,2015-02-10 09:29:00,21.05,36.0975,433.0,787.250000,0.005579,1
8140,2015-02-10 09:29:59,21.05,35.9950,433.0,789.500000,0.005563,1
8141,2015-02-10 09:30:59,21.10,36.0950,433.0,798.500000,0.005596,1
8142,2015-02-10 09:32:00,21.10,36.2600,433.0,820.333333,0.005621,1


Now that we have loaded the data into pandas, we now **load it into getML**:

In [5]:
data_train = getml.data.DataFrame.from_pandas(datatraining_pandas, name='data_train')

data_validate = getml.data.DataFrame.from_pandas(datatest_pandas, name='data_validate')

data_test = getml.data.DataFrame.from_pandas(datatest2_pandas, name='data_test')

## Annotating data

We need to assign **roles**.

To learn more about what roles do and why we need them, check out the [official documentation](https://docs.getml.com/latest/user_guide/annotating_data/annotating_data.html).

In [6]:
for df in [data_train, data_validate, data_test]:
    df.set_role(['Occupancy'], getml.data.roles.target)
    df.set_role(['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio'], getml.data.roles.numerical)
    df.set_role(['date'], getml.data.roles.time_stamp)

Time stamps in getML are always interpreted as **days**. But in this data set, the data is measured every minute. To make this a bit more readable, we define a simple helper function:

In [7]:
def minutes(x):
    """Helper function to convert minutes to days.
    
    Note that there are 60 minutes in an hour and 24 hours in a day.
    """
    return x / 60.0 / 24.0

Now we need to make this data set relational:

First, we add a **dummy join key**. The entire data set is actually one big time series - most other time series data seta contain more than just one series. So we can just add a constant value to the entire data frame.

Second, we add an **upper time stamp**. Upper time stamps impose an **upper limit** on all of our aggregations. In this case, we only want to aggregate the data over the **last 10 minutes**.

In effect, a simplified version of what we are going for is this:

```sql
CREATE TABLE FEATURE_1 AS
SELECT SOME_AGGREGATION( t2.some_column ) AS feature_1,
       t1.join_key,
       t1.date
FROM data_train t1
LEFT JOIN data_train t2
ON t1.join_key = t2.join_key
WHERE (...some conditions...) 
AND t2.date <= t1.date
AND t2.upper_time_stamp > t1.date 
GROUP BY t1.join_key,
         t1.date;```
         
This means that the feature engineering algorithm can only **aggregate values from the past** (```t2.date <= t1.date```), and only from the **last 10 minutes** (```t2.upper_time_stamp > t1.date```).

Upper time stamps are a but tricky to understand at first, but once you've wrapped your head around them, they are very powerful.

In [8]:
for df in [data_train, data_validate, data_test]:
    df.add(np.ones(len(df)), 'join_key', getml.data.roles.join_key)
    df.add(df["date"] + minutes(10),  "upper_time_stamp", getml.data.roles.time_stamp)

## Defining the data model

We need to build the data model. We do so by simply **joining the table onto itself** and imposing the conditions we have just discussed.

In [9]:
population = data_train.to_placeholder()

peripheral = data_train.to_placeholder()

population.join(
    peripheral, 
    join_key="join_key", 
    time_stamp="date",
    upper_time_stamp="upper_time_stamp"
)

## Setting the hyperparameters

We use a [MultirelModel](https://docs.getml.com/latest/api/getml.models.MultirelModel.html) for generating the features and an [XGBoostClassifier](https://docs.getml.com/latest/api/getml.predictors.XGBoostClassifier.html#getml.predictors.XGBoostClassifier) for feature selection and prediction.

We really don't spend a lot of effort on the hyperparameters and largely go with the default values. The only exception is that we add some regularization to the XGBoostClassifiers.

In [10]:
feature_selector = getml.predictors.XGBoostClassifier(reg_lambda=500)

predictor = getml.predictors.XGBoostClassifier(reg_lambda=500)

model = getml.models.MultirelModel(
        num_features=30,
        population=population,
        peripheral=[peripheral],
        loss_function=getml.models.loss_functions.CrossEntropyLoss(),
        feature_selector=feature_selector,
        predictor=predictor,
        seed=1706
)

## Fitting the model

We now fit the model. This should take well under one minute, depending on your computer.

In [11]:
model = model.fit(
    population_table=data_train,
    peripheral_tables=[data_train]
)

Loaded data. Features are now being trained...
Trained model.
Time taken: 0h:0m:27.504409



## Scoring the model

Let's see how well we did.

In [12]:
in_sample = model.score(population_table=data_train,
              peripheral_tables=[data_train]
             )
out_of_sample = model.score(population_table=data_test,
              peripheral_tables=[data_test]
             )

print("Accuracy (training): {:.5f}\nAUC (training): {:.5f}\n\nAccuracy (testing): {:.5f}\nAUC (testing): {:.5f}".format(
    in_sample['accuracy'][0], in_sample['auc'][0], out_of_sample['accuracy'][0], out_of_sample['auc'][0]))

Accuracy (training): 0.99460
AUC (training): 0.99899

Accuracy (testing): 0.99210
AUC (testing): 0.99743


In the [original paper](http://www.worldresearchlibrary.org/up_proc/pdf/568-148612088816-20.pdf), the authors tried several approaches. The **best out-of-sample values** of all the approaches they tried are the following:

* Accuracy (testing): 0.99061
* AUC (testing): 0.99574

Note that our results **outperform the best approach from the original paper**, both in terms of accuracy as well as AUC. This is despite the fact that we arguably **did not spend a lot of time on this**.

This demonstrates that relational learning is a **powerful tool for time series**.

## Studying the features

It is always a good idea to study the features the relational learning algorithm has extracted.

We can do so in the [feature view](https://docs.getml.com/latest/user_guide/getml_suite/monitor/models.html#feature-view).

We first look at the **most important feature**. This feature is negatively correlated with the target (a correlation coefficient of -92%) and accounts for **almost 64% of the total predictive power**.

![alt text](feature-1.png "FEATURE_1")

So what is it? Let's take a look at the SQL code:

```sql
CREATE TABLE FEATURE_1 AS
SELECT MIN( t1.date - t2.date ) AS feature_1,
       t1.join_key,
       t1.date
FROM (
     SELECT *,
            ROW_NUMBER() OVER ( ORDER BY join_key, date ASC ) AS rownum
     FROM data_train
) t1
LEFT JOIN data_train t2
ON t1.join_key = t2.join_key
WHERE (
   ( t2.Light > 458.172840 AND t1.Light > 502.583333 AND t2.Humidity > 26.403623 AND t2.Light <= 684.267857 )
OR ( t2.Light > 458.172840 AND t1.Light <= 502.583333 AND t2.Temperature > 22.264286 AND t2.Temperature > 22.315385 )
OR ( t2.Light > 458.172840 AND t1.Light <= 502.583333 AND t2.Temperature <= 22.264286 )
OR ( t2.Light <= 458.172840 AND t1.date - t2.date > 0.006185 )
OR ( t2.Light <= 458.172840 AND t1.date - t2.date <= 0.006185 AND t1.Light > 365.987395 )
OR ( t2.Light <= 458.172840 AND t1.date - t2.date <= 0.006185 AND t1.Light <= 365.987395 AND t1.Humidity > 37.597339 )
) AND t2.date <= t1.date
AND ( t2.upper_time_stamp > t1.date OR t2.upper_time_stamp IS NULL )
GROUP BY t1.rownum,
         t1.join_key,
         t1.date;
```

Simply speaking, this feature checks **how long ago the light was last switched on**. If that was relatively recent, it is more likely that the room is occupied. There are other factors, such as temperature and humidity that come into the equation as well, but simply speaking this is what the feature does.

The **second most important feature** has a strong positive correlation with the target (88%) and accounts for **13% of the predictive power**:

![alt text](feature-2.png "FEATURE_2")

```sql
CREATE TABLE FEATURE_2 AS
SELECT MAX( t2.Temperature ) AS feature_2,
       t1.join_key,
       t1.date
FROM (
     SELECT *,
            ROW_NUMBER() OVER ( ORDER BY join_key, date ASC ) AS rownum
     FROM data_train
) t1
LEFT JOIN data_train t2
ON t1.join_key = t2.join_key
WHERE (
   ( t1.Light > 365.007463 AND t1.Temperature > 22.625156 AND t1.Temperature > 22.659000 AND t1.Light <= 522.805556 )
OR ( t1.Light > 365.007463 AND t1.Temperature > 22.625156 AND t1.Temperature <= 22.659000 AND t2.Temperature <= 22.622222 )
OR ( t1.Light > 365.007463 AND t1.Temperature <= 22.625156 AND t1.Humidity > 18.924881 AND t1.date - t2.date <= 0.000661 )
) AND t2.date <= t1.date
AND ( t2.upper_time_stamp > t1.date OR t2.upper_time_stamp IS NULL )
GROUP BY t1.rownum,
         t1.join_key,
         t1.date;

```

Simply speaking, this feature is based on **the maximum temperature, given some other conditions**. The higher the temperature, the more likely it is that the room is occupied.

It is very unlikely that you would have written **features like this manually**. You might have written simpler features that are similar to this, but you have had to put in **more effort**.

Also note that some of the features are **columns directly taken from the original table**, such as *Light* and *CO2*. But these columns are **less correlated** and **less important** than the generated features, as we can see from the [model view](https://docs.getml.com/latest/user_guide/getml_suite/monitor/models.html#model-view).


![alt text](model-view.png "Model View")


## Conclusion

This notebook demonstrates that **relational learning is a powerful tool for time series**. We able to outperform the benchmarks for a scientific paper on a simple public domain time series data set using **relatively little effort**.

If you want to learn more, about getML, check out the [official documentation](https://getml.com/product).

To reproduce the results in this notebook, you can [download getML for free](https://docs.getml.com/latest/).
